In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
browser = webdriver.Chrome("/usr/local/bin/chromedriver")

In [3]:
df = pd.read_csv(r'data/city_codes.csv')

In [4]:
location = 'Delhi'
location = location.lower().capitalize()
locationCode = df['iata_code'][df[df['city'] == location].index.tolist()].tolist()[0]

In [5]:
checkin = '08/07/2020' #dd/mm/yyyy
tempDate = checkin.split('/')
checkin_url = tempDate[1] + tempDate[0] + tempDate[2]

In [6]:
checkout = '09/07/2020' #dd/mm/yyyy
tempDate = checkout.split('/')
checkout_url = tempDate[1] + tempDate[0] + tempDate[2]

In [7]:
browser.implicitly_wait(20)

In [8]:
url = 'https://www.makemytrip.com/hotels/hotel-listing/?checkin=' + checkin_url + '&checkout=' + checkout_url + '&locusId=CT' + locationCode + '&locusType=city&city=CT' + locationCode + '&country=IN&searchText=' + location + ''&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city'

In [9]:
browser.get(url)

In [11]:
# Scroll the page till bottom to get full data available in the DOM.
print ("Scrolling document upto bottom ...")
for j in range(1, 100):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Find the document body and get its inner HTML for processing in BeautifulSoup parser.
body = browser.find_element_by_tag_name("body").get_attribute("innerHTML")

Scrolling document upto bottom ...


In [31]:
#Trying infinite scroll
""""
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 20

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1
body = elem.get_attribute("innerHTML")
""""

In [12]:
print("Getting data from DOM ...")
soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup

Getting data from DOM ...


In [13]:
pHotelName = soupBody.findAll("p", {"class": "latoBlack font22 blackText appendBottom12"}) 

In [14]:
hotelsNameData=[]
for j in range(0, len(pHotelName)):
    hotelsNameData.append(pHotelName[j].text)

In [16]:
pHotelPrice = soupBody.findAll("p", {"class": "latoBlack font26 blackText appendBottom5"}) 

In [17]:
hotelsPriceData=[]
for j in range(0, len(pHotelPrice)):
    hotelsPriceData.append(pHotelPrice[j].text)

In [26]:
scraped_data = []

for i in range(len(hotelsPriceData)):

    # initialize the dictionary
    card_details = {}

    # add data to the dictionary
    card_details['hotel_name'] = hotelsNameData[i]
    card_details['room_price'] = hotelsPriceData[i]

    # append the scraped data to the list
    scraped_data.append(card_details)

# create a data frame from the list of dictionaries
dataFrame = pd.DataFrame.from_dict(scraped_data)

In [27]:
dataFrame.insert(0, 'location', location)
dataFrame.insert(0, 'index', range(0, len(dataFrame)))

In [28]:
dataFrame

,index,location,hotel_name,room_price
0,0,Delhi,Red Fox Hotel Delhi Airport,"INR2,631"
1,1,Delhi,The Piccadily,"INR4,550"
2,2,Delhi,The Park New Delhi,"INR5,499"
3,3,Delhi,Lemon Tree Premier Delhi Airport,"INR3,283"
4,4,Delhi,OYO 26648 The Kytes,"INR1,124"


In [30]:
json_records = dataFrame.to_json(r'result.json',orient ='records') 